In [1]:
%reset -f

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingClassifier, StackingClassifier
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models, optimizers
import os

## Загружаем данные

In [5]:
folder_path = r"D:\Programming\international_hack_data\preprocessed_data\selected_data.csv" # Путь к папке с CSV файлами

dataset = pd.read_csv(folder_path, index_col=0)
print(dataset.shape)
dataset.head()

(70000, 4)


,FrL,FrR,OcR,target
0,0.076875,0.151062,0.049438,0
1,0.050438,0.133375,0.028438,0
2,0.031438,0.122813,0.017063,0
3,0.031750,0.109500,0.007313,0
4,0.029750,0.106625,-0.036812,0


In [12]:
splitline = int(dataset.shape[0]*0.85)

X = dataset.drop(columns=['target'])
y = dataset.target

X_train, X_test = X[:splitline], X[splitline:]
y_train, y_test = y[:splitline], y[splitline:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((59500, 3), (10500, 3), (59500,), (10500,))

In [14]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Models

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, SimpleRNN, Dropout, Input
from sklearn.metrics import accuracy_score

In [21]:
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    
    # Сбрасываем индексы в y, чтобы они совпадали с индексами X
    y = y.reset_index(drop=True)
    for i in range(len(X) - seq_length + 1):
        X_seq.append(X[i:i + seq_length])
        y_seq.append(y[i + seq_length - 1])
        
    return np.array(X_seq), np.array(y_seq)

In [28]:
# Параметры
seq_length = 7
num_features = X_train.shape[1]

# Преобразуем тренировочные и тестовые данные
X_train_seq, y_train_seq = create_sequences(X_train, y_train, seq_length)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, seq_length)

In [37]:
model = Sequential([
    Input(shape=(seq_length, num_features)),
    Bidirectional(layers.LSTM(256, return_sequences=False)),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')
])

In [38]:
X_train_seq.shape, X_train.shape

((59494, 7, 3), (59500, 3))

In [43]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_test_seq, y_test_seq),
    epochs=40,
    batch_size=32)

Epoch 1/20


ValueError: in user code:

    File "C:\Users\Артём\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Артём\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Артём\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Артём\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 1055, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "C:\Users\Артём\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 1149, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "C:\Users\Артём\AppData\Roaming

In [42]:
model.save('first_model.h5')